In [1]:
import numpy as np
import pandas as pd

In [2]:
traindf = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')

In [3]:
# activity: 이벤트가 속한 활동 범주
# Nonproduction: 텍스트에 어떠한 변화도 일으키지 않는 이벤트
# Input: 에세이에 텍스트를 추가하는 이벤트
# Remove/Cut: 에세이에서 텍스트를 제거하는 이벤트
# Paste: 붙여넣기 입력을 통해 텍스트를 변경하는 이벤트
# Replace: 텍스트의 일부를 다른 문자열로 대체하는 이벤트
# Move From [x1, y1] To [x2, y2]: 텍스트의 일부를 x1, y1에서 x2, y2로 이동시키는 이벤트

In [ ]:
# 사용자의 텍스트 입력 및 편집 활동을 실시간으로 추적하여, 최종적으로 사용자가 작성한 에세이의 내용을 재구성

In [4]:
def getEssays(df):
    # Copy required columns
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']].copy()
    
# 변경 사항이 없는 텍스트 입력을 제거하십시오
# 참고: Shift 키가 예측 불가능하여 무시되었습니다

    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction'] 
    #Nonproduction: 텍스트에 어떠한 변화도 일으키지 않는 이벤트 제거

    # id' 열의 고유한 값들에 대해 개수를 세어 배열로 저장합니다.(unique 2471)
    valCountsArr = textInputDf['id'].value_counts(sort=False).values

    # Holds the final index of the previous Id
    lastIndex = 0

    # Holds all the essays
    essaySeries = pd.Series()

    # Fills essay series with essays
    for index, valCount in enumerate(valCountsArr): #index, 해당 id 갯수

        # Indexes down_time at current Id
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]

        # Update the last index
        lastIndex += valCount

        # Where the essay content will be stored
        essayText = ""

        
        #현재 'id'에 대한 행들을 반복하면서 'activity', 'cursor_position', 'text_change' 값을 가져옵니다.
        for Input in currTextInput.values:
            
            # Input[0] = activity
            # Input[2] = cursor_position
            # Input[3] = text_change
            
            #  activity에 따라 텍스트 어떻게 수정하는지 
            # 'Replace' 활동: 텍스트를 교체하고 에세이 텍스트를 업데이트
            # 'Paste' 활동: 텍스트를 붙여넣고 에세이 텍스트를 업데이트
            # 'Remove/Cut' 활동: 텍스트를 제거하고 에세이 텍스트를 업데이트
            # 'Move' 활동: 텍스트를 이동하고 에세이 텍스트를 업데이트
                        
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

                
            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

                
            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

                
            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]
                
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')
                
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]
                
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))

                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
                
                
            # If just input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]

            
        # Sets essay at index  
        essaySeries[index] = essayText
     
    
    # Sets essay series index to the ids
    essaySeries.index =  textInputDf['id'].unique()
    
    
    # Returns the essay series
    return essaySeries

In [5]:
%%time
essays = getEssays(traindf)

CPU times: user 8min 17s, sys: 3min 26s, total: 11min 43s
Wall time: 11min 44s


In [6]:
print(essays[0])

qqqqqqqqq qq qqqqq qq qqqq qqqq.  qqqqqq qqq qqqq qqqqqq qq qq qqqqq qq qqqq qqqqq qq qqqqqqqqq qqqqq qqqq qqqqq qqq qqqqqqqqq qqqqqqqqq qqqq.  qqqqqq qqq qqqqq qqq qqqqqqqqqqq qq qqq qqqqqqqqqq qqqqq, qqq qqqqq qqqqqq qq qq qqqq qqq qqqqqq qqqqqqq qq qqq qqqqqqqqqqq.  qqqqqqqq qq qqqqqqqqqq qqqq qqqq qqqqqqqqq qqq qqqqqqq qq qqqqqq qqqq qqq qqq qq qqqqqqqqq qq qq qqq qqqqq qqqqq qq qqq.

qq qq qqqq qqqq qqq qqqqqqqqq qqq qqqqqqq qq qqq qqqqq qqqqq, qq qq qqqqqq qqq qqq qqqqqqqq qqqqq qq qqq qqqqqqqqqqq qq qqqqqqqqq.  qqqqqqqqq qq qqq qqqqqqqq qqqq qq qqqq qq qqqqqqq qqqqq qqqqq, qqq qqqqqq qqqqq qqqqq qqq qqq qq qqq qqqqqqq qqqqqqq qqqq.  qqqq qqqqq qqqqq qqqq qqqq'qq qqqqq qqqqqqqqq qqqqq qqqqqqq qqqqqqq qqqqqqqqqq, qqqq qq qqqqqqqqqq qqqqqqq qqq qqqqqqq; qqqqqqq, qqqqq qqqqqqqq qqqqqq qqqqqqq qqqqqqq qqq qqqqq qqq qqq qqq qqqqqqq.  qqqq qqqqqqqqq qqqq qqq qqqq qqqq qqqqq qqqqqqqqqq qqqq qqqqq qqqqq.  qqq qqqqqqqqqq qq qqqqqqqq q qqqqqq, qqqqqqqq qqqq qqqq qqqqqqqqqq, qqq. qq qqqqq q

In [8]:
essays.head()

001519c8    qqqqqqqqq qq qqqqq qq qqqq qqqq.  qqqqqq qqq q...
0022f953    qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq...
0042269b    qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
0059420b    qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
0075873a    qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq...
dtype: object

In [9]:
essays.info()

<class 'pandas.core.series.Series'>
Index: 2471 entries, 001519c8 to fff05981
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
2471 non-null   object
dtypes: object(1)
memory usage: 103.1+ KB


In [10]:
essays.to_csv('train_essays')